# Comparación de análisis sociodemográfico de áreas de interés

This notebook takes as input gdfs generated on notebook 06_analisis_sociodemografico_poligono (which contain a summary of fields of interest from censo) and compares them.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


In [30]:
# Save csv final output?
analysis_output = True
analysis_output_dir = f'../../data/external/temporal_fromjupyter/arroyo_vivo/sociocomparison_arroyovivo.csv'

## Load data

In [23]:
aoi_list = ['poli','cuenca','div_01','div_02','div_03','div_04']
i = 0

for aoi in aoi_list:
    # Load gdf
    socio_gdf = pd.read_csv(f'../../data/external/temporal_fromjupyter/arroyo_vivo/censoanalysis_for_{aoi}.csv')

    # Format gdf
    socio_gdf.drop(columns='Unnamed: 0',inplace=True)
    socio_gdf.rename(columns={'data':'#'+aoi,
                              'wgt_%':'%'+aoi},inplace=True)

    # Merge gdf
    if i == 0:
        merged = socio_gdf[['indicador','weight','#'+aoi,'%'+aoi]]
    else:
        merged = merged.merge(socio_gdf[['indicador','#'+aoi,'%'+aoi]], on='indicador')

    i +=1

# Show
print(merged.shape)
merged.head(2)

(74, 14)


,indicador,weight,#poli,%poli,#cuenca,%cuenca,#div_01,%div_01,#div_02,%div_02,#div_03,%div_03,#div_04,%div_04
0,POBTOT,NaN,97554.0,NaN,104048.0,NaN,12348.0,NaN,25748.0,NaN,35674.0,NaN,31874.0,NaN
1,POBFEM,POBTOT,49160.0,50.392603,52327.0,50.291212,6541.0,52.972141,13239.0,51.417586,17650.0,49.475809,15650.0,49.09958


In [29]:
# Find zone of highest value for each row
comparing_values = ['%div_01','%div_02','%div_03','%div_04']

merged['highest'] = merged[comparing_values].idxmax(axis=1)

# Show
merged.head(37)

,indicador,weight,#poli,%poli,#cuenca,%cuenca,#div_01,%div_01,#div_02,%div_02,#div_03,%div_03,#div_04,%div_04,highest
0,POBTOT,NaN,97554.0,NaN,104048.0,NaN,12348.0,NaN,25748.0,NaN,35674.0,NaN,31874.0,NaN,NaN
1,POBFEM,POBTOT,49160.0,50.392603,52327.0,50.291212,6541.0,52.972141,13239.0,51.417586,17650.0,49.475809,15650.0,49.099580,%div_01
2,POBMAS,POBTOT,47826.0,49.025155,51150.0,49.160003,5769.0,46.720117,12430.0,48.275594,17895.0,50.162583,15856.0,49.745874,%div_03
3,P_0A2,POBTOT,2466.0,2.527831,2781.0,2.672805,288.0,2.332362,764.0,2.967221,1423.0,3.988899,343.0,1.076112,%div_03
4,P_0A2_F,POBFEM,911.0,1.853133,1023.0,1.955014,104.0,1.589971,296.0,2.235818,539.0,3.053824,98.0,0.626198,%div_03
5,P_0A2_M,POBMAS,931.0,1.946640,1089.0,2.129032,112.0,1.941411,280.0,2.252615,599.0,3.347304,117.0,0.737891,%div_03
6,P_3A5,POBTOT,3043.0,3.119298,3301.0,3.172574,358.0,2.899255,1032.0,4.008078,1552.0,4.350507,419.0,1.314551,%div_03
7,P_3A5_F,POBFEM,1217.0,2.475590,1325.0,2.532154,152.0,2.323804,461.0,3.482136,592.0,3.354108,151.0,0.964856,%div_02
8,P_3A5_M,POBMAS,1232.0,2.576005,1336.0,2.611926,155.0,2.686774,382.0,3.073210,685.0,3.827885,140.0,0.882947,%div_03
9,P_6A11,POBTOT,5673.0,5.815241,6381.0,6.132746,687.0,5.563654,1911.0,7.421936,2960.0,8.297359,932.0,2.924013,%div_03


In [28]:
# Show
merged.tail(37)

,indicador,weight,#poli,%poli,#cuenca,%cuenca,#div_01,%div_01,#div_02,%div_02,#div_03,%div_03,#div_04,%div_04,highest
37,P3YM_HLI_F,P_3YMAS_F,533.0,1.130675,662.0,1.353479,210.0,3.346614,192.0,1.560722,269.0,1.595019,14.0,0.098598,%div_01
38,P3YM_HLI_M,P_3YMAS_M,340.0,0.742764,427.0,0.896155,0.0,0.000000,149.0,1.294751,264.0,1.549660,18.0,0.125226,%div_03
39,PCON_DISC,POBTOT,3066.0,3.142875,3252.0,3.125481,105.0,0.850340,701.0,2.722542,1362.0,3.817907,1111.0,3.485600,%div_03
40,PCON_LIMI,POBTOT,8953.0,9.177481,9218.0,8.859373,475.0,3.846777,2097.0,8.144322,3460.0,9.698940,3266.0,10.246596,%div_04
41,P3A5_NOA,P_3A5,600.0,19.717384,607.0,18.388367,50.0,13.966480,109.0,10.562016,413.0,26.610825,44.0,10.501193,%div_03
42,P3A5_NOA_F,P_3A5_F,196.0,16.105177,184.0,13.886792,16.0,10.526316,25.0,5.422993,132.0,22.297297,18.0,11.920530,%div_03
43,P3A5_NOA_M,P_3A5_M,166.0,13.474026,167.0,12.500000,10.0,6.451613,36.0,9.424084,112.0,16.350365,9.0,6.428571,%div_03
44,P6A11_NOA,P_6A11,121.0,2.132910,136.0,2.131327,60.0,8.733624,23.0,1.203558,56.0,1.891892,3.0,0.321888,%div_01
45,P6A11_NOAF,P_6A11_F,21.0,0.887199,27.0,0.993377,10.0,3.322259,3.0,0.344037,14.0,1.114650,0.0,0.000000,%div_01
46,P6A11_NOAM,P_6A11_M,29.0,1.130604,33.0,1.121686,19.0,5.810398,6.0,0.668896,11.0,0.790230,0.0,0.000000,%div_01


In [31]:
if analysis_output:
    merged.to_csv(analysis_output_dir)